In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.impute import SimpleImputer

___
# Data Cleaning and Wrangling

### *Processing of the **Raw** Dataset*
___

In [2]:
#Load test dataset into variable for readability 
train_path = ("/Users/sa14/Desktop/Music Recommendation Algorithm with Unsupervised Learning/data/music_rec_trainset.csv")

#Load music recommendation training dataset csv as a dataframe
raw_df = pd.read_csv(train_path)

In [3]:
#Remove "Unnamed:0" column from raw dataframe - Unnamed:0 is an index error
raw_df.drop(raw_df.filter(regex="Unnamed").columns, axis=1, inplace=True)

#Verify "Unnamed:0" column removal
raw_df.head()

,artist_name,track_name,release_date,genre,lyrics,len,dating,violence,world/life,night/time,...,communication,obscene,music,movement/places,light/visual perceptions,family/spiritual,sadness,feelings,topic,age
0,mukesh,mohabbat bhi jhoothi,1950,pop,hold time feel break feel untrue convince spea...,95,0.000598,0.063746,0.000598,0.000598,...,0.263751,0.000598,0.039288,0.000598,0.000598,0.000598,0.380299,0.117175,sadness,1.0
1,frankie laine,i believe,1950,pop,believe drop rain fall grow believe darkest ni...,51,0.035537,0.096777,0.443435,0.001284,...,0.001284,0.001284,0.118034,0.001284,0.212681,0.051124,0.001284,0.001284,world/life,1.0
2,johnnie ray,cry,1950,pop,sweetheart send letter goodbye secret feel bet...,24,0.002770,0.002770,0.002770,0.002770,...,0.250668,0.002770,0.323794,0.002770,0.002770,0.002770,0.002770,0.225422,music,1.0
3,pérez prado,patricia,1950,pop,kiss lips want stroll charm mambo chacha merin...,54,0.048249,0.001548,0.001548,0.001548,...,0.001548,0.001548,0.001548,0.129250,0.001548,0.001548,0.225889,0.001548,romantic,1.0
4,giorgos papadopoulos,apopse eida oneiro,1950,pop,till darling till matter know till dream live ...,48,0.001350,0.001350,0.417772,0.001350,...,0.001350,0.001350,0.001350,0.001350,0.001350,0.029755,0.068800,0.001350,romantic,1.0


In [4]:
#Get features in raw dataset 
raw_df.columns

Index(['artist_name', 'track_name', 'release_date', 'genre', 'lyrics', 'len',
       'dating', 'violence', 'world/life', 'night/time', 'shake the audience',
       'family/gospel', 'romantic', 'communication', 'obscene', 'music',
       'movement/places', 'light/visual perceptions', 'family/spiritual',
       'sadness', 'feelings', 'topic', 'age'],
      dtype='object')

In [5]:
#store categorical features in a list called 'categorical_ids' then remove them from the dataset

categorical_ids_len = ['artist_name', 'track_name', 'genre', 'topic', 
                       'lyrics', 'len', 'release_date']
cleaned_numeric_dataset= raw_df.drop(columns=(categorical_ids_len))

#create final cleaned CSV file of only numeric features

cleaned_numeric_dataset.to_csv("cleaned_numeric_dataset.csv")

* #### The categorical features in the list categorical_ids_len were removed because they are non-numeric.

* #### Len, age, and release_date were dropped due to moderate to high correlations within those features and others.
    * #### Age & release_date have a perfect negative correlation where as the release date integer increases, the age (ranging from 0 to 1) decreases. (Ex. A song release in 1950 will have an age of 1. A song released in 2016 will have an age closer to 0.)
    * #### Len has a moderate correlation of 0.44 with the 'Obscene' topic 

___
### *Processing of the **Test** Dataset*
___

#### The algorithm created in model.ipynb will be applied to this dataset

In [6]:
#Load test dataset into variable for readability 
test_path = ("/Users/sa14/Desktop/Music Recommendation Algorithm with Unsupervised Learning/data/recommend_test_set.csv")

#Load music recommendation training dataset csv as a dataframe
test_df = pd.read_csv(test_path)

In [7]:
#Remove "Unnamed:0" column from raw dataframe
test_df.drop(test_df.filter(regex="Unnamed").columns, axis=1, inplace=True)

#Verify "Unnamed:0" column removal
test_df.head()

,artist_name,track_name,release_date,genre,lyrics,len,dating,violence,world/life,night/time,...,obscene,music,movement/places,light/visual perceptions,family/spiritual,like/girls,sadness,feelings,topic,age
0,godsmack,immune,1998,rock,come world society futher place home land deat...,74,0.000907,0.348191,0.375448,0.000907,...,0.000907,0.019389,0.000907,0.000907,0.000907,0.000907,0.000907,0.018854,world/life,0.314286
1,dennis brown,second chance,1993,reggae,maybe maybe treat good feel second best girl s...,43,0.001224,0.029943,0.001224,0.306688,...,0.001224,0.001224,0.001224,0.001224,0.001224,0.056842,0.001224,0.062092,night/time,0.385714
2,the black crowes,sister luck,1990,pop,worry sick eye hurt rest head life outside gir...,54,0.001120,0.482490,0.001120,0.001120,...,0.001120,0.001120,0.001120,0.078222,0.001120,0.051132,0.031571,0.202862,violence,0.428571
3,jerry lee lewis,your cheating heart,1960,pop,cheat heart weep sleep sleep come night cheat ...,25,0.204740,0.002506,0.002506,0.129818,...,0.002506,0.002506,0.002506,0.002506,0.002506,0.002506,0.474607,0.002506,sadness,0.857143
4,paul anka,eso beso,1966,pop,beso kiss beso kiss know samba bossanova close...,97,0.001170,0.001170,0.001170,0.050289,...,0.001170,0.001170,0.001170,0.314626,0.001170,0.053731,0.001170,0.001170,romantic,0.771429


In [8]:
#Get features in test dataset 
test_df.columns

Index(['artist_name', 'track_name', 'release_date', 'genre', 'lyrics', 'len',
       'dating', 'violence', 'world/life', 'night/time', 'shake the audience',
       'family/gospel', 'romantic', 'communication', 'obscene', 'music',
       'movement/places', 'light/visual perceptions', 'family/spiritual',
       'like/girls', 'sadness', 'feelings', 'topic', 'age'],
      dtype='object')

In [9]:
#store categorical features in a list called 'categorical_ids' then remove them from the dataset

testdf_cat_ids_len = ['artist_name', 'track_name', 'genre', 'topic', 'lyrics', 
                      'like/girls', 'len', 'release_date', 'age']
# "like/girls dropped because it is not in the raw dataset"

cleaned_rec_dataset= test_df.drop(columns=(testdf_cat_ids_len))

#create final cleaned CSV file of only numeric features

cleaned_rec_dataset.to_csv("rec_test_clean_set.csv")

* #### The "like/girls" feature was removed as it is not present in the raw dataset. 

* #### The other features in the testdf_cat_ids list will be reintroduced to categorize the output from training and testing the model.